In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
import pyspark.sql.functions as F

In [4]:
from pyspark.sql.types import *

In [5]:
import pandas as pd

In [6]:
sc = pyspark.SparkContext(appName="Pi")

In [9]:
sqlc = pyspark.sql.SQLContext(sc)

In [63]:
songs = pd.read_csv('../songdata.csv', usecols=['text'], nrows=3)

In [64]:
songs = sqlc.createDataFrame(songs)

In [65]:
songs.show(5)

+--------------------+
|                text|
+--------------------+
|Look at her face,...|
|Take it easy with...|
|I'll never know w...|
+--------------------+



In [66]:
def scanning(text, length):
    encoder_input = []
    decoder_input = []
    text_length = len(text)
    for i in range(0, text_length-2*length+1):
        encoder_input.append(list(text[i:i+length]))
        decoder_input.append(list(text[i+length:i+2*length]))
    return encoder_input, decoder_input

In [67]:
schema = StructType([
    StructField('for_encoder', ArrayType(ArrayType(StringType())), False),
    StructField('for_decoder', ArrayType(ArrayType(StringType())), False)
])

In [68]:
scanning_udf = F.udf(lambda text: scanning(text, 200), schema)

In [69]:
songs = songs.withColumn('lists', scanning_udf(songs.text))

In [70]:
songs.show(5)

+--------------------+--------------------+
|                text|               lists|
+--------------------+--------------------+
|Look at her face,...|[WrappedArray(Wra...|
|Take it easy with...|[WrappedArray(Wra...|
|I'll never know w...|[WrappedArray(Wra...|
+--------------------+--------------------+



In [54]:
songs = songs.withColumn('for_encoder', F.udf(lambda x: x[0])(songs.lists)).withColumn('for_decoder', F.udf(lambda x: x[1])(songs.lists))

In [55]:
songs.show(5)

+--------------------+--------------------+--------------------+--------------------+
|                text|               lists|         for_encoder|         for_decoder|
+--------------------+--------------------+--------------------+--------------------+
|Look at her face,...|[WrappedArray(Wra...|[[L, o, o, k,  , ...|[[k, e, s,  , m, ...|
|Take it easy with...|[WrappedArray(Wra...|[[T, a, k, e,  , ...|[[u, r,  , b, o, ...|
|I'll never know w...|[WrappedArray(Wra...|[[I, ', l, l,  , ...|[[g,  , h, e, r, ...|
+--------------------+--------------------+--------------------+--------------------+



In [71]:
songs.dtypes

[('text', 'string'),
 ('lists',
  'struct<for_encoder:array<array<string>>,for_decoder:array<array<string>>>')]

In [21]:
res = songs.agg(F.collect_list(songs.for_encoder).alias('for_encoder')).collect()

In [30]:
songs_rdd = songs.rdd

In [34]:
songs_rdd.reduce(lambda x, y: x['for_encoder'] + y['for_encoder'])

TypeError: string indices must be integers